# Part 1

In [32]:
import re

class TangramPuzzle:
    def __init__(self, filename):
        with open(filename) as f:
            self.content = f.readlines()
        self.transformations = {}
        self.transformations = self.parse_transformations()
        
    def parse_transformations(self):
        for line in self.content:
            if "PieceTangram" not in line:
                continue
            else:
                shape_name, x_flip, y_flip, rotate = self.analyse_line(line)
                if shape_name in self.transformations:
                    shape_name = shape_name.replace("1", "2")
                self.transformations[shape_name] = {"x_flip": x_flip, "y_flip": y_flip, "rotate": rotate}
        return self.transformations

    def analyse_line(self, line):
        type_to_name = {
            "TangGrandTri": "Large triangle 1",
            "TangMoyTri": "Medium triangle",
            "TangPetTri": "Small triangle 1",
            "TangCar": "Square",
            "TangPara": "Parallelogram" 
        }
        shape_type = re.findall(r"\{([^{}]*)\}", line)[-1]
        shape_name = type_to_name.get(shape_type)
        if shape_name is None:
            raise ValueError(f"Unknown shape type: {shape_type}")
        data = re.findall(r"<([^<>]*)>", line)
        if data:
            data = data[0].replace(" ", "").split(",")
        x_flip, y_flip, rotate = self.cal_rotate(data)
        return shape_name, x_flip, y_flip, rotate

    def cal_rotate(self, data):
        x_flip, y_flip, rotate = False, False, 0
        for line in data:
            name, value = line.split("=")
            if name == "xscale" and value == "-1":
                x_flip = True
            elif name == "yscale" and value == "-1":
                y_flip = True
            elif name == "rotate":
                rotate = int(value)
        if x_flip and y_flip:
            rotate = (rotate + 180) % 360
            x_flip = False
            y_flip = False
        return x_flip, y_flip, rotate
    
    def __str__(self):
        pass


In [33]:
T = TangramPuzzle("kangaroo.tex")
# T = TangramPuzzle("cat.tex")
# T = TangramPuzzle("goose.tex")
for piece in T.transformations:
    print(f'{piece:16}', T.transformations[piece], sep = ": ")

Large triangle 1: {'x_flip': False, 'y_flip': False, 'rotate': 0}
Large triangle 2: {'x_flip': False, 'y_flip': False, 'rotate': 180}
Small triangle 1: {'x_flip': False, 'y_flip': False, 'rotate': 45}
Small triangle 2: {'x_flip': False, 'y_flip': False, 'rotate': -90}
Parallelogram   : {'x_flip': True, 'y_flip': False, 'rotate': -45}
Medium triangle : {'x_flip': False, 'y_flip': False, 'rotate': 0}
Square          : {'x_flip': False, 'y_flip': False, 'rotate': -45}


# Part 2

In [38]:
from fractions import Fraction

class Point:
    def __init__(self, x_expr, y_expr):
        self.x_rational, self.x_sqrt2 = self._parse_expr(x_expr)
        self.y_rational, self.y_sqrt2 = self._parse_expr(y_expr)

    def _parse_expr(self, expr):
        tokens = expr.replace(" ", "").replace("-", " -").replace("+", " +").split()
        rational = Fraction(0)
        sqrt2 = Fraction(0)

        for token in tokens:
            if "sqrt(2)" not in token:
                rational += Fraction(token)
            else:
                coeff = token.replace("sqrt(2)", "").replace("*", "")
                if coeff == "" or coeff == "+":
                    sqrt2 += Fraction(1)
                elif coeff == "-":
                    sqrt2 -= Fraction(1)
                else:
                    sqrt2 += Fraction(coeff)
        return rational, sqrt2
    
    def _format(self, rational, sqrt2):
        parts = []
        negative = sqrt2 < 0
        if rational:
            parts.append(str(rational))
        if sqrt2:
            if rational:
                sqrt2 = abs(sqrt2)
            if sqrt2.denominator != 1:
                sqrt_part = f"({sqrt2})√2"
            elif sqrt2.numerator != 1 and sqrt2.numerator != -1:
                sqrt_part = f"{sqrt2}√2"
            elif sqrt2.numerator == 1:
                sqrt_part = "√2"
            elif sqrt2.numerator == -1:
                sqrt_part = "-√2"
            if rational == 0:
                parts.append(sqrt_part)
            else:
                parts.append(f"- {sqrt_part}" if negative else f"+ {sqrt_part}")
        return " ".join(parts) if parts else "0"

    def rotate_90(self):
        self.x_rational, self.y_rational = -self.y_rational, self.x_rational
        self.x_sqrt2, self.y_sqrt2 = -self.y_sqrt2, self.x_sqrt2

    def flip(self, x_flip = False, y_flip = False):
        if x_flip:
            self.x_rational = -self.x_rational
            self.x_sqrt2 = -self.x_sqrt2
        if y_flip:
            self.y_rational = -self.y_rational
            self.y_sqrt2 = -self.y_sqrt2
    
    def move(self, anchor):
        self.x_rational += anchor.x_rational
        self.y_rational += anchor.y_rational
        self.x_sqrt2 += anchor.x_sqrt2
        self.y_sqrt2 += anchor.y_sqrt2

    def __str__(self):
        return f"({self._format(self.x_rational, self.x_sqrt2)}, {self._format(self.y_rational, self.y_sqrt2)})"

In [39]:
point = Point("-2.5 -1.5* sqrt(2)", " + 2 *sqrt(2)")
print(point)

(-5/2 - (3/2)√2, 2√2)


# Final

In [83]:
import re
from fractions import Fraction

base_shape = {
    "Large triangle":{
        0: [("0", "0"), ("2", "2"), ("2", "0")],
        45: [("0", "0"), ("0", "2 * sqrt(2)"), ("sqrt(2)", "sqrt(2)")],
    },
    "Small triangle":{
        0: [("0", "0"), ("1", "1"), ("1", "0")],
        45: [("0", "0"), ("0", "sqrt(2)"), ("0.5 * sqrt(2)", "0.5 * sqrt(2)")],
    },
    "Medium triangle":{
        0: [("0", "0"), ("1", "1"), ("2", "0")],
        45: [("0", "0"), ("0", "sqrt(2)"), ("sqrt(2)", "sqrt(2)")],
    },
    "Square":{
        0: [("0", "0"), ("1", "0"), ("1", "1"), ("0", "1")],
        45: [("0", "0"), ("- 0.5 * sqrt(2)", "0.5 * sqrt(2)"), ("0", "sqrt(2)"), ("0.5 * sqrt(2)", "0.5 * sqrt(2)")],
    },
    "Parallelogram":{
        0: [("0", "0"), ("1", "1"), ("2", "1"), ("1", "0")],
        45: [("0", "0"), ("0", "sqrt(2)"), ("0.5 * sqrt(2)", "1.5 * sqrt(2)"), ("0.5 * sqrt(2)", "0.5 * sqrt(2)")],
    },
}

class Point:
    def __init__(self, x_expr, y_expr):
        self.x_rational, self.x_sqrt2 = self._parse_expr(x_expr)
        self.y_rational, self.y_sqrt2 = self._parse_expr(y_expr)

    def _parse_expr(self, expr):
        tokens = expr.replace(" ", "").replace("-", " -").replace("+", " +").split()
        rational = Fraction(0)
        sqrt2 = Fraction(0)

        for token in tokens:
            if "sqrt(2)" not in token:
                rational += Fraction(token)
            else:
                coeff = token.replace("sqrt(2)", "").replace("*", "")
                if coeff == "" or coeff == "+":
                    sqrt2 += Fraction(1)
                elif coeff == "-":
                    sqrt2 -= Fraction(1)
                else:
                    sqrt2 += Fraction(coeff)
        return rational, sqrt2
    
    def _format(self, rational, sqrt2):
        parts = []
        if rational:
            parts.append(str(rational))
        if sqrt2:
            negative = sqrt2 < 0
            if rational:
                sqrt2 = abs(sqrt2)
            if sqrt2.denominator != 1:
                sqrt_part = f"({sqrt2})√2"
            elif sqrt2.numerator != 1 and sqrt2.numerator != -1:
                sqrt_part = f"{sqrt2}√2"
            elif sqrt2.numerator == 1:
                sqrt_part = "√2"
            elif sqrt2.numerator == -1:
                sqrt_part = "-√2"
            if rational == 0:
                parts.append(sqrt_part)
            else:
                parts.append(f"- {sqrt_part}" if negative else f"+ {sqrt_part}")
        return " ".join(parts) if parts else "0"

    def __str__(self):
        return f"({self._format(self.x_rational, self.x_sqrt2)}, {self._format(self.y_rational, self.y_sqrt2)})"
    
    def rotate_90(self):
        self.x_rational, self.y_rational = -self.y_rational, self.x_rational
        self.x_sqrt2, self.y_sqrt2 = -self.y_sqrt2, self.x_sqrt2

    def flip(self, x_flip = False, y_flip = False):
        if x_flip:
            self.x_rational = -self.x_rational
            self.x_sqrt2 = -self.x_sqrt2
        if y_flip:
            self.y_rational = -self.y_rational
            self.y_sqrt2 = -self.y_sqrt2
    
    def move(self, anchor):
        self.x_rational += anchor.x_rational
        self.y_rational += anchor.y_rational
        self.x_sqrt2 += anchor.x_sqrt2
        self.y_sqrt2 += anchor.y_sqrt2
    
    def approx_x(self):
        return float(self.x_rational) + float(self.x_sqrt2) * (2 ** 0.5)

    def approx_y(self):
        return float(self.y_rational) + float(self.y_sqrt2) * (2 ** 0.5)

class TangramPuzzle:
    def __init__(self, filename):
        with open(filename) as f:
            self.content = f.readlines()
        self.vertices = []
        self.transformations = {}
        self.transformations = self.parse_transformations()

        self.vertices.sort(key = lambda shape: (-shape[1][0].approx_y(), shape[1][0].approx_x(), shape[1][1].approx_x()))
        
    def parse_transformations(self):
        transformations = {}
        for line in self.content:
            if "PieceTangram" not in line:
                continue
            else:
                shape_name, x_flip, y_flip, rotate = self.analyse_line(line)
                if shape_name in transformations:
                    shape_name = shape_name.replace("1", "2")
                transformations[shape_name] = {"x_flip": x_flip, "y_flip": y_flip, "rotate": rotate}
        return self.transformations

    def analyse_line(self, line):
        type_to_name = {
            "TangGrandTri": "Large triangle 1",
            "TangMoyTri": "Medium triangle",
            "TangPetTri": "Small triangle 1",
            "TangCar": "Square",
            "TangPara": "Parallelogram" 
        }
        shape_type = re.findall(r"\{([^{}]*)\}", line)[-1]
        shape_name = type_to_name.get(shape_type)
        data = re.findall(r"<([^<>]*)>", line)

        if data:
            data = data[0].replace(" ", "").split(",")
        x_flip, y_flip, rotate = self.cal_rotate(data)

        point_data = re.findall(r"\{([^{}]*)\}", line)[: 2]
        self.compute_vertices(point_data, shape_name.replace("1", "").rstrip(), x_flip, y_flip, rotate)
        return shape_name, x_flip, y_flip, rotate

    def cal_rotate(self, data):
        x_flip, y_flip, rotate = False, False, 0
        for line in data:
            name, value = line.split("=")
            if name == "xscale" and value == "-1":
                x_flip = True
            elif name == "yscale" and value == "-1":
                y_flip = True
            elif name == "rotate":
                rotate = int(value)
        if x_flip and y_flip:
            rotate = (rotate + 180) % 360
            x_flip = False
            y_flip = False
        return x_flip, y_flip, rotate
    
    def compute_vertices(self, point_data, shape_name, x_flip, y_flip, rotate):
        anchor = Point(point_data[0], point_data[1])
        base_angle = 0 if rotate % 90 == 0 else 45
        base_coords = base_shape[shape_name][base_angle]

        steps = (rotate - base_angle) // 90
        points = [Point(x, y) for x, y in base_coords]

        for point in points:
            for _ in range(steps):
                point.rotate_90()
            point.flip(x_flip = x_flip, y_flip = y_flip)
            point.move(anchor)
        
        if x_flip != y_flip:
            points = points[::-1]
        
        top_left = min(points, key = lambda p: (-p.approx_y(), p.approx_x()))
        index = points.index(top_left)
        ordered_points = points[index:] + points[:index]
        self.vertices.append((shape_name, ordered_points))
        

    def __str__(self):
        lines = []
        for shape_name, points in self.vertices:
            line = f"{shape_name:15}: {', '.join(str(p) for p in points)}"
            lines.append(line)
        return "\n".join(lines)


In [84]:
print(TangramPuzzle("kangaroo.tex"))

Small triangle : (3, 4), (3, 3), (2, 3)
Large triangle : (2, 2), (2, 0), (0, 0)
Small triangle : (2, -1/2 + √2), (2 + (1/2)√2, -1/2 + (1/2)√2), (2, -1/2)
Parallelogram  : ((-1/2)√2, -2 + (3/2)√2), (0, -2 + √2), (0, -2), ((-1/2)√2, -2 + (1/2)√2)
Large triangle : (0, 0), (2, 0), (0, -2)
Square         : (1 - (1/2)√2, -1 + (1/2)√2), (1, -1), (1 - (1/2)√2, -1 - (1/2)√2), (1 - √2, -1)
Medium triangle: (1 - (1/2)√2, -1 - (1/2)√2), (2 - (1/2)√2, -2 - (1/2)√2), ((-1/2)√2, -2 - (1/2)√2)
